In [ ]:
# !pip uninstall tidyzoning -y
# !pip install git+https://github.com/HOUPU1993/tidyzoning.git

Found existing installation: tidyzoning 0.1.0
Uninstalling tidyzoning-0.1.0:
  Successfully uninstalled tidyzoning-0.1.0
  Cloning https://github.com/HOUPU1993/tidyzoning.git to c:\users\admin\appdata\local\temp\pip-req-build-yagkkfwn
  Resolved https://github.com/HOUPU1993/tidyzoning.git to commit 8c3fb0945838de2b26229a4f8930e4ea04c450f8
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tidyzoning: filename=tidyzoning-0.1.0-py3-none-any.whl size=2642 sha256=0b3f93d5d0c4f47df0d0e67f9058a33afb3e00ab56c0f13a4eed449afe19072f
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-ec54wjo6\wheels\f4\29\51\e431eb95e0bf6589310375ce483daada9c916a6f65f3b65a8f
Successfully built tidyzoning


  Running command git clone --filter=blob:none --quiet https://github.com/HOUPU1993/tidyzoning.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-yagkkfwn'


In [ ]:
# from tidyzoning import custom_formula

# # made a test
# print(custom_formula(2, 5))  # x=2, constant=5
# print(custom_formula(0, 1))  # x=0, constant=1
# print(custom_formula(-3, 8)) # x=-3, constant=8

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import Point

In [ ]:
tidybuilding = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data/Tidybuildings/tidybuilding_2family.geojson")
tidyparcel = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data/Test_Parcels/combined_testparcel_100per_correct.shp")
tidydistrict = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data//Tidyzoning/tidyzoning_Haltom_City.geojson")

tidydistrict = tidydistrict.to_crs(tidyparcel.crs)
tidybuilding = tidybuilding.to_crs(tidyparcel.crs)

In [ ]:
tidyparcel

In [ ]:
import geopandas as gpd
from shapely.geometry import box

tidyparcel = gpd.read_file(r"C:/Users\Admin/Desktop/zoning_data/Test_Data/Tarrant_County_combined_parcel.geojson")
tidyparcel = tidyparcel.to_crs(tidydistrict.crs)

district_bbox = tidydistrict.total_bounds  # Returns [minx, miny, maxx, maxy]
bounding_box = box(district_bbox[0], district_bbox[1], district_bbox[2], district_bbox[3])  # Create bounding box geometry

# Step 2: Convert the bounding box into a GeoDataFrame for easier spatial operations
bbox_gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs=tidydistrict.crs)

# Step 3: Find parcels that intersect with the bounding box
tidyparcel = tidyparcel[tidyparcel.intersects(bounding_box)]

In [21]:
tidyparcel

,OBJECTID,Situs_Addr,RP,parcel_lab,Found_Matc,match_road,shape_inde,50_thresho,num_edges,angle_diff,original_i,new_index,shared_sid,parcel_bea,road_beari,angle,distance_t,side,geometry
0,48,2321 N BEACH ST,P,regular corner parcel,1,N Beach St,1.222134,0,4,2.491631,1,0,0,0.009221,1.586372,89.635914,43.678329,Exterior side,"LINESTRING (-10830411.144 3868123.459, -108303..."
1,48,2321 N BEACH ST,P,regular corner parcel,1,N Beach St,1.222134,0,4,2.491631,1,0,0,-1.511734,1.586372,2.491631,15.565707,front,"LINESTRING (-10830359.122 3868123.939, -108303..."
2,48,2321 N BEACH ST,P,regular corner parcel,1,N Beach St,1.222134,0,4,2.491631,1,0,1,3.130028,1.586372,88.444992,42.001352,Interior side,"LINESTRING (-10830351.930 3868002.311, -108304..."
3,48,2321 N BEACH ST,P,regular corner parcel,1,N Beach St,1.222134,0,4,2.491631,1,0,1,1.570243,1.586372,0.924128,70.634370,rear,"LINESTRING (-10830411.211 3868002.997, -108304..."
4,451,4101 CARNATION AVE,R,regular inside parcel,1,Carnation Ave,1.229169,0,4,1.016347,1,0,1,1.583615,0.006137,89.617161,27.322890,Interior side,"LINESTRING (-10829959.495 3866831.725, -108299..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17331,2836787,BLANDIN ST,R,regular inside parcel,1,Blandin St,1.135663,0,4,0.924055,2,0,0,1.552463,1.568591,0.924055,11.673686,front,"LINESTRING (-10832081.622 3867718.774, -108320..."
17332,2851618,804 BLANDIN ST,R,regular inside parcel,1,Blandin St,1.126165,0,4,2.650760,2,0,1,-0.011120,1.511444,87.236527,20.976098,Interior side,"LINESTRING (-10832070.141 3865423.013, -108320..."
17333,2851618,804 BLANDIN ST,R,regular inside parcel,1,Blandin St,1.126165,0,4,2.650760,2,0,1,-1.582126,1.511444,2.751506,35.408516,rear,"LINESTRING (-10832042.710 3865422.708, -108320..."
17334,2851618,804 BLANDIN ST,R,regular inside parcel,1,Blandin St,1.126165,0,4,2.650760,2,0,1,3.029722,1.511444,86.990902,22.830237,Interior side,"LINESTRING (-10832043.056 3865392.168, -108320..."


In [22]:
tidydistrict

,dist_info,lot_constraints,structure_constraints,other_constraints,geometry
0,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829182.585 3866215.182, -10..."
1,"{'dist_name': 'Single-Family Residential', 'di...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...","MULTIPOLYGON (((-10829727.287 3866115.173, -10..."
2,"{'dist_name': 'Manufactured Housing', 'dist_ab...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...",None
3,"{'dist_name': 'Duplex Residential', 'dist_abbr...","{'lot_size': [{'unit': 'acres', 'use_name': ['...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10829844.874 3866386.842, -10..."
4,"{'dist_name': 'Townhouse Residential', 'dist_a...","{'lot_size': [{'unit': 'acres', 'use_name': '1...","{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': '1...",None
5,"{'dist_name': 'Multiple Family Residential', '...",[ ],"{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...",None
6,"{'dist_name': 'Multiple Family Residential', '...",[ ],"{'setback_front': [{'unit': 'feet', 'use_name'...","{'parking': [{'unit': 'stalls', 'use_name': ['...","MULTIPOLYGON (((-10827770.055 3873318.245, -10..."
7,"{'dist_name': 'Office', 'dist_abbr': 'O', 'dis...",[ ],[ ],[ ],"MULTIPOLYGON (((-10830277.539 3866854.440, -10..."
8,"{'dist_name': 'Commercial', 'dist_abbr': 'C-1'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828680.964 3866302.949, -10..."
9,"{'dist_name': 'Commercial', 'dist_abbr': 'C-2'...",[ ],[ ],[ ],"MULTIPOLYGON (((-10828928.601 3866165.418, -10..."
